In [411]:
from IslandTime import Workflow, retrieve_island_info, TimeSeriesCoastSat, update_data_map, update_results_map, plot_shoreline_transects, Segmentation
import matplotlib.pyplot as plt
import Rbeast as rb
from datetime import datetime
import os
import numpy as np
from tqdm import tqdm
import matplotlib
import pickle
from coastsatmaster.coastsat import SDS_preprocess, SDS_tools
from osgeo import gdal
import skimage.exposure as exposure
import shapely
import pyproj
import geopandas as gpd
import skimage.transform as transform
from arosics import COREG
import scipy.stats as stats

matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Choose island and image

In [396]:
island = "Hulhudhoo (South Maalhosmadulu)"
country = "Maldives"

island_info = retrieve_island_info(island, country, verbose=False)

# Functions

In [397]:
def _image_stretch(im):
    # Rescale image for visualisation
    if np.ndim(im) > 2:
        for i in range(np.shape(im)[2]):
            im_no_nan = im[:, :, i][~np.isnan(im[:, :, i])]
            im[:, :, i] = exposure.rescale_intensity(im[:, :, i], in_range=(im_no_nan.min(), im_no_nan.max()))
    
    else:
        im_no_nan = im[~np.isnan(im)]
        im = exposure.rescale_intensity(im, in_range=(im_no_nan.min(), im_no_nan.max()))
    
    return im

def get_pol(island_info, nir, ndvi, ndwi, rgb, centre_image, ic, md, find_pol):
    seg = Segmentation(island_info, list_sat=['S2', 'L8'])

    labels_opt = ['NIR', 'NDVI', 'NDWI']
    stacked_array_opt = [nir, ndvi, ndwi]

    list_dict_polygons_segmented = []

    # Loop through labels (only NIR, NDVI and NDWI for now)
    for label, im in zip(labels_opt, stacked_array_opt):
        dict_polygons_segmented, X_peaks, X, histogram_distribution = seg._band_segmentation(im, 
                                                                                                rgb, 
                                                                                                label, 
                                                                                                segmented_image=None, 
                                                                                                cmap=None, 
                                                                                                classes_to_consider=None,
                                                                                                plot_results=False, 
                                                                                                animation=False,
                                                                                                find_polygons=find_pol,
                                                                                                missing_data_testing=md)
        list_dict_polygons_segmented.append(dict_polygons_segmented)

    # Find optimal polygon
    res_optimal = seg._find_optimal_polygon(list_dict_polygons_segmented, centre_image, rgb, plot_results=True, ignore_centre=ic)
    
    return res_optimal

def _coregistration_PSS(rgb, nir, visual_inspection=False, return_geotransform=False):
    # Define list of parameters for co-registration
    ws_list = [1000, 600, 500, 400, 300, 200, 100, 50, 10]
    res_list = []

    for ws in ws_list:
        try:
            CRL = COREG(rgb, nir, align_grids=True, max_shift=100, ws=(ws, ws), max_iter=100, q=True)
            res = CRL.correct_shifts()
            res_list.append(res)
        except:
            continue

    # Nothing has been found
    if len(res_list) == 0:
        return nir

    if visual_inspection:
        # Create figure
        _, ax = plt.subplots(1, len(res) + 1)

        for i, res in enumerate(res_list):
            ax[i].imshow(rgb[:, :, 0], cmap='gray')
            ax[i].imshow(res['arr_shifted'], alpha=0.4)
            ax[i].set_title('ws = {}'.format(ws_list[i]))
        ax[-1].imshow(rgb[:, :, 0], cmap='gray')
        ax[-1].imshow(nir, alpha=0.4)
        ax[-1].set_title('before correction')
        plt.show()
        # plt.close()

    # Take the first result (typically the best result)
    nirc_corrected = res_list[0]['arr_shifted']

    if return_geotransform:
        geotransform_corrected = res_list[0]['GeoArray_shifted'].geotransform

        return nirc_corrected, geotransform_corrected

    return nirc_corrected


In [593]:
keyyy = 'polygon_NIR_opt'

# Sentinel-2 and Landsat-8/9

In [613]:
# List of images
list_images = ['2021-07-22-05-25-11_L8', '2022-04-04-05-24-58_L8', '2021-08-09-05-37-42_S2', '2022-04-06-05-37-34_S2']

path_files = os.path.join(os.getcwd(), 'data', 'coastsat_data', '{}_{}'.format(island, country))

settings_LS = island_info['timeseries_coastsat']['settings']

ff = list_images[0]
sat = ff.split('_')[-1]

# Load metadata (CoastSat)
with open(os.path.join(path_files, '{}_{}_metadata.pkl'.format(island, country)), 'rb') as f:
    metadata = pickle.load(f)

# File path and names from CoastSat folder
filepath = SDS_tools.get_filepath(settings_LS['inputs'], sat)
filenames = metadata[sat]['filenames']

filename = filenames[np.argwhere([ff in filenamee for filenamee in filenames])[0][0]]

# Define date from filename
str_filename = filename.split('-')
year, month, day = str_filename[0], str_filename[1], str_filename[2]

# Next steps are taken from CoastSat
# Get file name
fn = SDS_tools.get_filenames(filename, filepath, sat)
fn = [fn[i].replace('c:\\Users\\mp222', 'C:\\Users\\myriampe') for i in range(len(fn))]

# Retrieve information about image
#try:
im_ms, georef, cloud_mask, im_extra, im_QA, im_nodata = SDS_preprocess.preprocess_single(fn, sat, settings_LS['cloud_mask_issue'], settings_LS['pan_off'], 'C02')

#except:
    #continue

# Compute cloud_cover percentage (with no data pixels)
cloud_cover_combined = np.divide(sum(sum(cloud_mask.astype(int))), (cloud_mask.shape[0] * cloud_mask.shape[1]))

# Remove no data pixels from the cloud mask (for example L7 bands of no data should not be accounted for)
cloud_mask_adv = np.logical_xor(cloud_mask, im_nodata)

# Compute updated cloud cover percentage (without no data pixels)
cloud_cover = np.divide(sum(sum(cloud_mask_adv.astype(int))), (sum(sum((~im_nodata).astype(int)))))

# Get images
blue = im_ms[:, :, 0] * ~cloud_mask
green = im_ms[:, :, 1] * ~cloud_mask 
red = im_ms[:, :, 2] * ~cloud_mask
nir = im_ms[:, :, 3] * ~cloud_mask
swir = im_ms[:, :, 4] * ~cloud_mask

# Create RGB image
rgb = np.dstack((red, green, blue))

# Calculate NDVI and NDWI
ndvi = SDS_tools.nd_index(nir, red, cloud_mask)
ndwi = SDS_tools.nd_index(green, nir, cloud_mask)

image_epsg = metadata[sat]['epsg'][np.argwhere([ff in filenamee for filenamee in filenames])[0][0]]

# Source and target coordinate reference systems
src_crs = pyproj.CRS('EPSG:4326')
tgt_crs = pyproj.CRS('EPSG:{}'.format(image_epsg))

# Define transformer
transformer = pyproj.Transformer.from_crs(src_crs, tgt_crs, always_xy=True)

latitude = island_info['spatial_reference']['latitude']
longitude = island_info['spatial_reference']['longitude']

# Transform latitude and longitude to x and y
xx, yy = transformer.transform(longitude, latitude)

# Find indices of x and y
idx_x = int((xx - georef[0]) / georef[1])
idx_y = int((yy - georef[3]) / georef[5])

# Centre of the image based on indices
centre_image = shapely.geometry.Point(idx_x, idx_y)

pol_full = get_pol(island_info, nir, ndvi, ndwi, rgb, centre_image, ic=False, md=0., find_pol=False)

# transform

fig, ax = plt.subplots()
ax.imshow(_image_stretch(rgb), extent=[georef[0], georef[0] + georef[1] * rgb.shape[1], georef[3] + georef[5] * rgb.shape[0], georef[3]])

# Make affine transformation matrix
aff_mat = np.array([[georef[1], georef[2], georef[0]],
                    [georef[4], georef[5], georef[3]],
                    [0, 0, 1]])

# Create affine transformation
tform = transform.AffineTransform(aff_mat)

# Transform polygon
x_poly, y_poly = pol_full[keyyy].exterior.xy
tmp = np.column_stack((x_poly, y_poly))
points_converted = tform(tmp)
poly_image_crs = shapely.geometry.LineString(points_converted)

dict_pol = {}
# Transform polygon
x_poly, y_poly = pol_full['polygon_NIR_opt'].exterior.xy
tmp = np.column_stack((x_poly, y_poly))
points_converted = tform(tmp)
poly_image_crs = shapely.geometry.LineString(points_converted)
dict_pol['polygon_NIR_opt'] = poly_image_crs

# Transform polygon
x_poly, y_poly = pol_full['polygon_NIR_otsu'].exterior.xy
tmp = np.column_stack((x_poly, y_poly))
points_converted = tform(tmp)
poly_image_crs = shapely.geometry.LineString(points_converted)
dict_pol['polygon_NIR_otsu'] = poly_image_crs

# Transform polygon
x_poly, y_poly = pol_full['polygon_NDVI_opt'].exterior.xy
tmp = np.column_stack((x_poly, y_poly))
points_converted = tform(tmp)
poly_image_crs = shapely.geometry.LineString(points_converted)
dict_pol['polygon_NDVI_opt'] = poly_image_crs

# Transform polygon
x_poly, y_poly = pol_full['polygon_NDWI_opt'].exterior.xy
tmp = np.column_stack((x_poly, y_poly))
points_converted = tform(tmp)
poly_image_crs = shapely.geometry.LineString(points_converted)
dict_pol['polygon_NDWI_opt'] = poly_image_crs

gpd.GeoSeries(poly_image_crs).plot(ax=ax, edgecolor='red', facecolor='none', linewidth=2)

<Axes: >

In [595]:
# Transform polygon
x_poly, y_poly = pol_full['polygon_NIR_opt'].exterior.xy
tmp = np.column_stack((x_poly, y_poly))
points_converted = tform(tmp)
poly_image_crs = shapely.geometry.LineString(points_converted)

# Transform polygon
x_poly, y_poly = pol_full['polygon_NIR_otsu'].exterior.xy
tmp = np.column_stack((x_poly, y_poly))
points_converted = tform(tmp)
poly_image_crs = shapely.geometry.LineString(points_converted)

# Transform polygon
x_poly, y_poly = pol_full['polygon_NDVI_opt'].exterior.xy
tmp = np.column_stack((x_poly, y_poly))
points_converted = tform(tmp)
poly_image_crs = shapely.geometry.LineString(points_converted)

# Transform polygon
x_poly, y_poly = pol_full['polygon_NDWI_opt'].exterior.xy
tmp = np.column_stack((x_poly, y_poly))
points_converted = tform(tmp)
poly_image_crs = shapely.geometry.LineString(points_converted)

# gpd.GeoSeries(poly_image_crs).plot(ax=ax, edgecolor='red', facecolor='none', linewidth=2)

In [542]:
plt.imshow(_image_stretch(rgb))

In [596]:
poll_cov = {}
poll_cov['Optimised polygon'] = poly_image_crs

# PlanetScope

In [544]:
filenames_PSS[2]

'20210719_051252_1002_3B_AnalyticMS_SR_clip.tif'

In [597]:
from xml.dom import minidom
from osgeo import gdal, osr

# Define path for PSS images
filepath_PSS = os.path.join(os.getcwd(), 'data', 'coastsat_data', '{}_{}'.format(island, country), 'PSScene', 'Hulhudhoo_(July_August_2021)_psscene_analytic_sr_udm2', 'PSScene')
filenames_PSS = [f for f in os.listdir(filepath_PSS) if f.endswith('_AnalyticMS_SR_clip.tif')]

filename_PSS = filenames_PSS[2]

# Find corresponding xml file
corresponding_xml_file = filename_PSS.replace('_AnalyticMS_SR_clip.tif', '_AnalyticMS_metadata_clip.xml')
xmldoc = minidom.parse(os.path.join(filepath_PSS, '202107~3.XML'))
nodes = xmldoc.getElementsByTagName("ps:bandSpecificMetadata")

# XML parser refers to bands by numbers 1-4
coeffs = {}
for node in nodes:
    bn = node.getElementsByTagName("ps:bandNumber")[0].firstChild.data
    if bn in ['1', '2', '3', '4']:
        i = int(bn)
        value = node.getElementsByTagName("ps:reflectanceCoefficient")[0].firstChild.data
        coeffs[i] = float(value)
    
# Read .tif file
im_ms = gdal.Open(os.path.join(filepath_PSS, filename_PSS), gdal.GA_ReadOnly)

# Extract geotransform and bands and multiply by corresponding coefficients
geotransform_PSS = im_ms.GetGeoTransform()
blue_PSS = im_ms.GetRasterBand(1).ReadAsArray() * coeffs[1]
green_PSS = im_ms.GetRasterBand(2).ReadAsArray() * coeffs[2]
red_PSS = im_ms.GetRasterBand(3).ReadAsArray() * coeffs[3]
nir_PSS = im_ms.GetRasterBand(4).ReadAsArray() * coeffs[4]

# Replace 0 values with nan
blue_PSS = np.where(blue_PSS == 0., np.nan, blue_PSS)
green_PSS = np.where(green_PSS == 0., np.nan, green_PSS)
red_PSS = np.where(red_PSS == 0., np.nan, red_PSS)
nir_PSS = np.where(nir_PSS == 0., np.nan, nir_PSS)

# Create RGB image
rgb_PSS = np.dstack((red_PSS, green_PSS, blue_PSS))

# Co-registration of NIR band
nir_corrected = _coregistration_PSS(rgb_PSS, nir_PSS, visual_inspection=True)
nir_PSS = nir_corrected

# Calculate NDVI and NDWI
ndvi_PSS = (nir_corrected - red_PSS) / (nir_corrected + red_PSS)
ndwi_PSS = (green_PSS - nir_corrected) / (green_PSS + nir_corrected)

georef_PSS = im_ms.GetGeoTransform()
projection_PSS = osr.SpatialReference(wkt=im_ms.GetProjection())
image_epsg_PSS = int(projection.GetAttrValue('AUTHORITY', 1))

# Source and target coordinate reference systems
src_crs = pyproj.CRS('EPSG:4326')
tgt_crs = pyproj.CRS('EPSG:{}'.format(image_epsg))

# Define transformer
transformer = pyproj.Transformer.from_crs(src_crs, tgt_crs, always_xy=True)

latitude = island_info['spatial_reference']['latitude']
longitude = island_info['spatial_reference']['longitude']

# Transform latitude and longitude to x and y
xx, yy = transformer.transform(longitude, latitude)

# Find indices of x and y
idx_x = int((xx - georef_PSS[0]) / georef_PSS[1])
idx_y = int((yy - georef_PSS[3]) / georef_PSS[5])

# Centre of the image based on indices
centre_image_PSS = shapely.geometry.Point(idx_x, idx_y)

pol_PSS = get_pol(island_info, nir_PSS, ndvi_PSS, ndwi_PSS, rgb_PSS, centre_image_PSS, ic=False, md=0., find_pol=True)

fig, ax = plt.subplots()
ax.imshow(_image_stretch(rgb_PSS), extent=[georef_PSS[0], georef_PSS[0] + georef_PSS[1] * rgb_PSS.shape[1], georef_PSS[3] + georef_PSS[5] * rgb_PSS.shape[0], georef_PSS[3]])

# Make affine transformation matrix
aff_mat = np.array([[georef_PSS[1], georef_PSS[2], georef_PSS[0]],
                    [georef_PSS[4], georef_PSS[5], georef_PSS[3]],
                    [0, 0, 1]])

# Create affine transformation
tform = transform.AffineTransform(aff_mat)

# Transform polygon
x_poly, y_poly = pol_PSS[keyyy].exterior.xy
tmp = np.column_stack((x_poly, y_poly))
points_converted = tform(tmp)
poly_image_crs_PSS = shapely.geometry.LineString(points_converted)

gpd.GeoSeries(poly_image_crs_PSS).plot(ax=ax, edgecolor='red', facecolor='none', linewidth=2)

poll_cov['Optimised polygon PlanetScope'] = poly_image_crs_PSS

# Other comparisons

In [598]:
pol_half_top = get_pol(island_info, nir, ndvi, ndwi, rgb, centre_image, ic=True, md='top', find_pol=False)
pol_half_bottom = get_pol(island_info, nir, ndvi, ndwi, rgb, centre_image, ic=True, md='bottom', find_pol=False)
pol_half_left = get_pol(island_info, nir, ndvi, ndwi, rgb, centre_image, ic=True, md='left', find_pol=False)
pol_half_right = get_pol(island_info, nir, ndvi, ndwi, rgb, centre_image, ic=True, md='right', find_pol=False)

In [357]:
pol_full = get_pol(island_info, nir, ndvi, ndwi, rgb, centre_image, ic=False, md=0., find_pol=False)
pol_half_top = get_pol(island_info, nir[:np.shape(nir)[0]//2+1, :], ndvi[:np.shape(ndvi)[0]//2+1, :], ndwi[:np.shape(ndwi)[0]//2+1, :], rgb[:np.shape(rgb)[0]//2+1, :], centre_image, ic=True, md=0., find_pol=False)
pol_half_bottom = get_pol(island_info, nir[np.shape(nir)[0]//2:, :], ndvi[np.shape(ndvi)[0]//2:, :], ndwi[np.shape(ndwi)[0]//2:, :], rgb[np.shape(rgb)[0]//2:, :], centre_image, ic=True, md=0., find_pol=False)
pol_half_left = get_pol(island_info, nir[:, :np.shape(nir)[1]//2+1], ndvi[:, :np.shape(ndvi)[1]//2+1], ndwi[:, :np.shape(ndwi)[1]//2+1], rgb[:, :np.shape(rgb)[1]//2+1], centre_image, ic=True, md=0., find_pol=False)
pol_half_right = get_pol(island_info, nir[:, np.shape(nir)[1]//2:], ndvi[:, np.shape(ndvi)[1]//2:], ndwi[:, np.shape(ndwi)[1]//2:], rgb[:, np.shape(rgb)[1]//2:], centre_image, ic=True, md=0., find_pol=False)

In [599]:
# pol_full = get_pol(island_info, nir, ndvi, ndwi, rgb, centre_image, ic=False, md=0., find_pol=False)
pol_01 = get_pol(island_info, nir, ndvi, ndwi, rgb, centre_image, ic=False, md=0.1, find_pol=False)
pol_02 = get_pol(island_info, nir, ndvi, ndwi, rgb, centre_image, ic=False, md=0.2, find_pol=False)
pol_03 = get_pol(island_info, nir, ndvi, ndwi, rgb, centre_image, ic=False, md=0.3, find_pol=False)
pol_04 = get_pol(island_info, nir, ndvi, ndwi, rgb, centre_image, ic=False, md=0.4, find_pol=False)
pol_05 = get_pol(island_info, nir, ndvi, ndwi, rgb, centre_image, ic=False, md=0.5, find_pol=False)
pol_06 = get_pol(island_info, nir, ndvi, ndwi, rgb, centre_image, ic=False, md=0.6, find_pol=False)
pol_07 = get_pol(island_info, nir, ndvi, ndwi, rgb, centre_image, ic=False, md=0.7, find_pol=False)
pol_08 = get_pol(island_info, nir, ndvi, ndwi, rgb, centre_image, ic=False, md=0.8, find_pol=False)
pol_09 = get_pol(island_info, nir, ndvi, ndwi, rgb, centre_image, ic=False, md=0.9, find_pol=False)

In [600]:
fig, ax = plt.subplots()
ax.imshow(_image_stretch(rgb), extent=[georef[0], georef[0] + georef[1] * rgb.shape[1], georef[3] + georef[5] * rgb.shape[0], georef[3]])

# Make affine transformation matrix
aff_mat = np.array([[georef[1], georef[2], georef[0]],
                    [georef[4], georef[5], georef[3]],
                    [0, 0, 1]])

# Create affine transformation
tform = transform.AffineTransform(aff_mat)

# Transform polygon
x_poly, y_poly = pol_full[keyyy].exterior.xy
tmp = np.column_stack((x_poly, y_poly))
points_converted = tform(tmp)
poly_image_crs = shapely.geometry.Polygon(points_converted)

pol_conv = {}
for p, l in zip([pol_01, pol_02, pol_03, pol_04, pol_05, pol_06, pol_07, pol_half_top, pol_half_bottom, pol_half_left, pol_half_right], ['10% of data removed', '20% of data removed', '30% of data removed', '40% of data removed', '50% of data removed', '60% of data removed', '70% of data removed', '50% of data removed (top)', '50% of data removed (bottom)', '50% of data removed (left)', '50% of data removed (right)']):
    x_poly, y_poly = p[keyyy].exterior.xy
    tmp = np.column_stack((x_poly, y_poly))
    points_converted = tform(tmp)
    poly_image_crs = shapely.geometry.LineString(points_converted)
    pol_conv[l] = poly_image_crs
    poll_cov[l] = poly_image_crs
    gpd.GeoSeries(poly_image_crs).plot(ax=ax)

In [612]:
fig, ax = plt.subplots()
ax.imshow(_image_stretch(rgb), extent=[georef[0], georef[0] + georef[1] * rgb.shape[1], georef[3] + georef[5] * rgb.shape[0], georef[3]])

list_color = matplotlib.cm.get_cmap('tab20').colors
for i, pol in enumerate(poll_cov.keys()):
    gpd.GeoSeries(poll_cov[pol]).plot(ax=ax, edgecolor=list_color[i], facecolor='none', linewidth=1, label=pol)
    if i == 0:
        polygon = shapely.geometry.Polygon(poll_cov[pol].coords)   
        polygon_buffer = polygon.buffer(5)
        polygon_buffer_d = polygon.buffer(-5)
        pol_diff = polygon_buffer.difference(polygon_buffer_d)
        # gpd.GeoSeries(polygon_buffer).plot(ax=ax, edgecolor=list_color[i], facecolor='none', linewidth=1, linestyle='--')
        # gpd.GeoSeries(polygon_buffer_d).plot(ax=ax, edgecolor=list_color[i], facecolor='none', linewidth=1, linestyle='--')
        gpd.GeoSeries(pol_diff).plot(ax=ax, edgecolor=list_color[i], facecolor=list_color[i], linewidth=1, linestyle='--', alpha=0.5)


plt.legend()

# Errors

In [602]:
# Define projection
src_crs = pyproj.CRS('EPSG:3857')
tgt_crs = pyproj.CRS('EPSG:{}'.format(image_epsg))

# Create transformer
transformer = pyproj.Transformer.from_crs(src_crs, tgt_crs, always_xy=True)
# Transform transects
transects_image_crs = {}
for key_transect, transect in island_info['spatial_reference']['transects'].items():
    transect_x_transformed, transect_y_transformed = transformer.transform(transect[:, 0], transect[:, 1])
    transects_image_crs[key_transect] = np.column_stack((transect_x_transformed, transect_y_transformed))


# # Make affine transformation matrix
# aff_mat = np.array([[georef[1], georef[2], georef[0]],
#                     [georef[4], georef[5], georef[3]],
#                     [0, 0, 1]])

# # Create affine transformation
# tform = transform.AffineTransform(aff_mat)

# # Transform polygon
# tmp = np.column_stack((x_poly, y_poly))
# points_converted = tform(tmp)
# poly_image_crs = shapely.geometry.Polygon(points_converted)

position_on_transect = {}

for key in poll_cov.keys():

    # Get coordinates of polygon (exterior)
    x_poly, y_poly  = poll_cov[key].coords.xy

    # Calculate position on transect
    position_on_transect[key] = {}

    for key_transect, transect in transects_image_crs.items():
        
        # Calculate intersection
        intersection = poll_cov[key].intersection(shapely.geometry.LineString(transect))

        # Calculate distance from bottom of transect
        point_transect = shapely.geometry.Point(transect[0, 0], transect[0, 1])

        if type(intersection) == shapely.geometry.LineString:
            position_on_transect[key][key_transect] = np.nan

        elif type(intersection) == shapely.geometry.Point:
            position_on_transect[key][key_transect] = point_transect.distance(intersection)

        if type(intersection) == shapely.geometry.MultiPoint:
            dist = [point_transect.distance(i) for i in intersection.geoms]
            position_on_transect[key][key_transect] = dist[np.argmin(dist)]

In [606]:
# Define projection
src_crs = pyproj.CRS('EPSG:3857')
tgt_crs = pyproj.CRS('EPSG:{}'.format(image_epsg))

# Create transformer
transformer = pyproj.Transformer.from_crs(src_crs, tgt_crs, always_xy=True)
# Transform transects
transects_image_crs = {}
for key_transect, transect in island_info['spatial_reference']['transects'].items():
    transect_x_transformed, transect_y_transformed = transformer.transform(transect[:, 0], transect[:, 1])
    transects_image_crs[key_transect] = np.column_stack((transect_x_transformed, transect_y_transformed))


# # Make affine transformation matrix
# aff_mat = np.array([[georef[1], georef[2], georef[0]],
#                     [georef[4], georef[5], georef[3]],
#                     [0, 0, 1]])

# # Create affine transformation
# tform = transform.AffineTransform(aff_mat)

# # Transform polygon
# tmp = np.column_stack((x_poly, y_poly))
# points_converted = tform(tmp)
# poly_image_crs = shapely.geometry.Polygon(points_converted)

position_on_transect = {}

for key in dict_pol.keys():

    # Get coordinates of polygon (exterior)
    x_poly, y_poly  = dict_pol[key].coords.xy

    # Calculate position on transect
    position_on_transect[key] = {}

    for key_transect, transect in transects_image_crs.items():
        
        # Calculate intersection
        intersection = dict_pol[key].intersection(shapely.geometry.LineString(transect))

        # Calculate distance from bottom of transect
        point_transect = shapely.geometry.Point(transect[0, 0], transect[0, 1])

        if type(intersection) == shapely.geometry.LineString:
            position_on_transect[key][key_transect] = np.nan

        elif type(intersection) == shapely.geometry.Point:
            position_on_transect[key][key_transect] = point_transect.distance(intersection)

        if type(intersection) == shapely.geometry.MultiPoint:
            dist = [point_transect.distance(i) for i in intersection.geoms]
            position_on_transect[key][key_transect] = dist[np.argmin(dist)]

In [608]:
def calc_CI(data):

    # Confidence level (e.g., 0.95 for 95%)
    confidence_level = 0.95

    # Calculate sample mean and standard deviation
    mean = np.mean(data)
    std_dev = np.std(data, ddof=1)

    # Sample size
    n = len(data)

    # Standard error
    se = std_dev / np.sqrt(n)

    # Find the critical value (t-distribution if n <= 30, otherwise z-distribution)
    if n > 30:
        critical_value = stats.norm.ppf((1 + confidence_level) / 2)
    else:
        critical_value = stats.t.ppf((1 + confidence_level) / 2, df=n-1)

    # Margin of error
    margin_of_error = critical_value * se

    # Confidence interval
    confidence_interval = (mean - margin_of_error, mean + margin_of_error)

    return confidence_interval, margin_of_error, mean

va = []
me = []
for key in transects_image_crs.keys():
    # print('Transect: {}'.format(key))
    _, ci, mean = calc_CI([position_on_transect[keyy][key] for keyy in position_on_transect.keys()])
    va.append(ci)
    me.append(mean)

calc_CI(va)

((6.387127682914846, 9.334668743553802), 1.4737705303194777, 7.860898213234324)

In [611]:

uncertainties = np.array([5.22, 6.3])
SE_mean = np.sqrt(np.sum(0.5 * uncertainties**2))
SE_mean

5.785257124795751

# Other

In [37]:
# Transform polygon
x_poly, y_poly = pol_full['polygon_NIR_otsu'].exterior.xy
tmp = np.column_stack((x_poly, y_poly))
points_converted = tform(tmp)
poly_image_crs = shapely.geometry.LineString(points_converted)

# Transform half top polygon
x_poly, y_poly = pol_half_top['polygon_NIR_otsu'].exterior.xy
tmp = np.column_stack((x_poly, y_poly))
points_converted = tform(tmp)
poly_image_crs_top = shapely.geometry.LineString(points_converted)

# Transform half bottom polygon
x_poly, y_poly = pol_half_bottom['polygon_NIR_otsu'].exterior.xy
tmp = np.column_stack((x_poly, y_poly))
points_converted = tform_bottom(tmp)
poly_image_crs_bottom = shapely.geometry.LineString(points_converted)

# Transform half left polygon
x_poly, y_poly = pol_half_left['polygon_NIR_otsu'].exterior.xy
tmp = np.column_stack((x_poly, y_poly))
points_converted = tform(tmp)
poly_image_crs_left = shapely.geometry.LineString(points_converted)

# Transform half right polygon
x_poly, y_poly = pol_half_right['polygon_NIR_otsu'].exterior.xy
tmp = np.column_stack((x_poly, y_poly))
points_converted = tform_right(tmp)
poly_image_crs_right = shapely.geometry.LineString(points_converted)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(_image_stretch(rgb), extent=[georef[0], georef[0] + georef[1] * rgb.shape[1], georef[3] + georef[5] * rgb.shape[0], georef[3]])

# Make affine transformation matrix
aff_mat = np.array([[georef[1], georef[2], georef[0]],
                    [georef[4], georef[5], georef[3]],
                    [0, 0, 1]])

aff_mat_top = aff_mat.copy()
aff_mat_bottom = aff_mat.copy()

# convert georef to half top image
aff_mat_top[1, 2] = aff_mat[1, 2] + np.shape(rgb)[0]//2 * georef[5]

# convert georef to half bottom image
aff_mat_bottom[1, 2] = aff_mat[1, 2] + np.shape(rgb)[0]//2 * georef[5]

# convert georef to half left image
aff_mat_left = aff_mat.copy()
aff_mat_left[0, 2] = aff_mat[0, 2] + np.shape(rgb)[1]//2 * georef[1]

# convert georef to half right image
aff_mat_right = aff_mat.copy()
aff_mat_right[0, 2] = aff_mat[0, 2] + np.shape(rgb)[1]//2 * georef[1]

# Create affine transformation
tform = transform.AffineTransform(aff_mat)
tform_top = transform.AffineTransform(aff_mat_top)
tform_bottom = transform.AffineTransform(aff_mat_bottom)
tform_left = transform.AffineTransform(aff_mat_left)
tform_right = transform.AffineTransform(aff_mat_right)

keyyy = 'polygon_NIR_otsu'

# Transform polygon
x_poly, y_poly = pol_full[keyyy].exterior.xy
tmp = np.column_stack((x_poly, y_poly))
points_converted = tform(tmp)
poly_image_crs = shapely.geometry.LineString(points_converted)

# Transform half top polygon
x_poly, y_poly = pol_half_top[keyyy].exterior.xy
tmp = np.column_stack((x_poly[:-1], y_poly[:-1]))
points_converted = tform(tmp)
poly_image_crs_top = shapely.geometry.LineString(points_converted)

# Transform half bottom polygon
x_poly, y_poly = pol_half_bottom[keyyy].exterior.xy
tmp = np.column_stack((x_poly[:-1], y_poly[:-1]))
points_converted = tform_bottom(tmp)
poly_image_crs_bottom = shapely.geometry.LineString(points_converted)

# Transform half left polygon
x_poly, y_poly = pol_half_left[keyyy].exterior.xy
tmp = np.column_stack((x_poly[:-1], y_poly[:-1]))
points_converted = tform(tmp)
poly_image_crs_left = shapely.geometry.LineString(points_converted)

# Transform half right polygon
x_poly, y_poly = pol_half_right[keyyy].exterior.xy
tmp = np.column_stack((x_poly[:-1], y_poly[:-1]))
points_converted = tform_right(tmp)
poly_image_crs_right = shapely.geometry.LineString(points_converted)

poll_cov['50% of data removed (top)'] = poly_image_crs_top
poll_cov['50% of data removed (bottom)'] = poly_image_crs_bottom
poll_cov['50% of data removed (left)'] = poly_image_crs_left
poll_cov['50% of data removed (right)'] = poly_image_crs_right

# first point full poly
# x_poly, y_poly = poly_image_crs.exterior.xy
# # ax.scatter(x_poly[29], y_poly[29], s=100, c='b')

# # first point full poly
# x_poly, y_poly = poly_image_crs_top.exterior.xy
# # ax.scatter(x_poly[0], y_poly[0], s=100, c='r')

# # first point full poly
# x_poly, y_poly = poly_image_crs_bottom.exterior.xy
# # ax.scatter(x_poly[0], y_poly[0], s=100, c='r')

# # first point full poly
# x_poly, y_poly = poly_image_crs_left.exterior.xy
# # ax.scatter(x_poly[0], y_poly[0], s=100, c='r')

# # first point full poly
# x_poly, y_poly = poly_image_crs_right.exterior.xy
# ax.scatter(x_poly[0], y_poly[0], s=100, c='r')

gpd.GeoSeries(poly_image_crs).plot(ax=ax)
gpd.GeoSeries(poly_image_crs_top).plot(ax=ax, color='r')
gpd.GeoSeries(poly_image_crs_bottom).plot(ax=ax, color='g')
gpd.GeoSeries(poly_image_crs_left).plot(ax=ax, color='y')
gpd.GeoSeries(poly_image_crs_right).plot(ax=ax, color='m')

In [38]:
keyyy = 'polygon_opt'

# Transform polygon
x_poly, y_poly = pol_full[keyyy].exterior.xy
tmp = np.column_stack((x_poly, y_poly))
points_converted = tform(tmp)
poly_image_crs = shapely.geometry.LineString(points_converted)

# Transform half right polygon
x_poly, y_poly = pol_05[keyyy].exterior.xy
tmp = np.column_stack((x_poly, y_poly))
points_converted = tform(tmp)
poly_image_crs_05 = shapely.geometry.LineString(points_converted)

In [39]:
def calc_err(pol1, pol2):

    # Convert polygons to a list of points
    
    points1 = [shapely.geometry.Point(coord) for coord in pol1.coords]
    points2 = [shapely.geometry.Point(coord) for coord in pol2.coords]
    
    fig, ax = plt.subplots()
    # gpd.GeoSeries(pol1).plot(ax=ax)
    # gpd.GeoSeries(pol2).plot(ax=ax, color='r')

    first_point2 = points2[0]

    # Calculate the distance between each point
    distances = []
    for p1 in points1:
        distances.append(p1.distance(first_point2))

    arg_start = np.argmin(distances)
    print(arg_start)

    # compare full and half top
    # points1s = points1[arg_start:] + list(np.flip(points1[:arg_start]))
    points1s = points1[arg_start:] + points1[:arg_start+1]
    # plot points

    # linestring = shapely.geometry.LineString([p.coords[0] for p in points1[arg_start:]])
    # linestring2 = shapely.geometry.LineString([p.coords[0] for p in np.flip(points1[:arg_start+1])])
    linestring3 = shapely.geometry.LineString([p.coords[0] for p in points1s])
    linestring4 = shapely.geometry.LineString([p.coords[0] for p in points2])

    # gpd.GeoSeries(linestring).plot(ax=ax)
    # gpd.GeoSeries(linestring2).plot(ax=ax, color='g')
    gpd.GeoSeries(linestring3).plot(ax=ax, color='r')
    gpd.GeoSeries(linestring4).plot(ax=ax, color='b')
    # gpd.GeoSeries(shapely.geometry.MultiPoint(points1[arg_start:arg_start+2])).plot(ax=ax)
    # gpd.GeoSeries(shapely.geometry.MultiPoint(np.flip(points1[:arg_start-1]))).plot(ax=ax, color='g')
    # gpd.GeoSeries(shapely.geometry.MultiPoint(points1s)).plot(ax=ax, color='r')

    diff_top = [points1s[i].distance(points2[i]) for i in range(len(points2))] 
    diff_plot = [shapely.geometry.LineString([points1s[i], points2[i]]) for i in range(len(points2))]

    # plot differences
    for d in diff_plot:
        gpd.GeoSeries(d).plot(ax=ax, color='y')

    fig, ax = plt.subplots()
    ax.plot(diff_top)
    
    return np.mean(diff_top), points1, points2

In [245]:
def calc_err(pol1, pol2):
    
    fig, ax = plt.subplots()
    coords_pol1 = np.array(pol1.coords)
    coords_pol2 = np.array(pol2.coords)

    gpd.GeoSeries(pol1).plot(ax=ax, color='r')
    gpd.GeoSeries(pol2).plot(ax=ax, color='b')

    # Loop in coords of pol1
    distances = []
    diff_plot = []
    if len(coords_pol2) > len(coords_pol1):
        for i in range(len(coords_pol1)):
            # find the closest point in pol2
            dist = np.sqrt((coords_pol1[i][0] - coords_pol2[:, 0])**2 + (coords_pol1[i][1] - coords_pol2[:, 1])**2)
            distances.append(np.min(dist))
        
            diff_plot.append(shapely.geometry.LineString([coords_pol1[i], coords_pol2[np.argmin(dist)]]))
    
    else:
        for i in range(len(coords_pol2)):
            # find the closest point in pol2
            dist = np.sqrt((coords_pol2[i][0] - coords_pol1[:, 0])**2 + (coords_pol2[i][1] - coords_pol1[:, 1])**2)
            distances.append(np.min(dist))
        
            diff_plot.append(shapely.geometry.LineString([coords_pol2[i], coords_pol1[np.argmin(dist)]]))
    
    figg, axx = plt.subplots()
    axx.plot(distances)

    for d in diff_plot:
        gpd.GeoSeries(d).plot(ax=ax, color='y')
    
    return np.mean(distances), np.std(distances), coords_pol1, coords_pol2

In [50]:
# Make affine transformation matrix
aff_mat = np.array([[georef[1], georef[2], georef[0]],
                    [georef[4], georef[5], georef[3]],
                    [0, 0, 1]])

aff_mat_top = aff_mat.copy()
aff_mat_bottom = aff_mat.copy()

# convert georef to half top image
aff_mat_top[1, 2] = aff_mat[1, 2] + np.shape(rgb)[0]//2 * georef[5]

# convert georef to half bottom image
aff_mat_bottom[1, 2] = aff_mat[1, 2] + np.shape(rgb)[0]//2 * georef[5]

# convert georef to half left image
aff_mat_left = aff_mat.copy()
aff_mat_left[0, 2] = aff_mat[0, 2] + np.shape(rgb)[1]//2 * georef[1]

# convert georef to half right image
aff_mat_right = aff_mat.copy()
aff_mat_right[0, 2] = aff_mat[0, 2] + np.shape(rgb)[1]//2 * georef[1]

# Create affine transformation
tform = transform.AffineTransform(aff_mat)
tform_top = transform.AffineTransform(aff_mat_top)
tform_bottom = transform.AffineTransform(aff_mat_bottom)
tform_left = transform.AffineTransform(aff_mat_left)
tform_right = transform.AffineTransform(aff_mat_right)

keyyy = 'polygon_NIR_otsu'

# Transform polygon
x_poly, y_poly = pol_full[keyyy].exterior.xy
tmp = np.column_stack((x_poly, y_poly))
points_converted = tform(tmp)
poly_image_crs = shapely.geometry.LineString(points_converted)

# Transform half top polygon
x_poly, y_poly = pol_half_top[keyyy].exterior.xy
tmp = np.column_stack((x_poly[:-1], y_poly[:-1]))
points_converted = tform(tmp)
poly_image_crs_top = shapely.geometry.LineString(points_converted)

# Transform half bottom polygon
x_poly, y_poly = pol_half_bottom[keyyy].exterior.xy
tmp = np.column_stack((x_poly[:-1], y_poly[:-1]))
points_converted = tform_bottom(tmp)
poly_image_crs_bottom = shapely.geometry.LineString(points_converted)

# Transform half left polygon
x_poly, y_poly = pol_half_left[keyyy].exterior.xy
tmp = np.column_stack((x_poly[:-1], y_poly[:-1]))
points_converted = tform(tmp)
poly_image_crs_left = shapely.geometry.LineString(points_converted)

# Transform half right polygon
x_poly, y_poly = pol_half_right[keyyy].exterior.xy
tmp = np.column_stack((x_poly[:-1], y_poly[:-1]))
points_converted = tform_right(tmp)
poly_image_crs_right = shapely.geometry.LineString(points_converted)

pol_conv_top = {'pol_full': poly_image_crs, 'pol_half_top': poly_image_crs_top, 'pol_half_bottom': poly_image_crs_bottom, 'pol_half_left': poly_image_crs_left, 'pol_half_right': poly_image_crs_right}

In [54]:
keyyy = 'polygon_NIR_otsu'

pol_conv = {}
for p, l in zip([pol_full, pol_01, pol_02, pol_03, pol_04, pol_05, pol_06, pol_07, pol_08, pol_09], ['pol_full', 'pol_01', 'pol_02', 'pol_03', 'pol_04', 'pol_05', 'pol_06', 'pol_07', 'pol_08', 'pol_09']):
    print(l)
    if p is None:
        continue
    x_poly, y_poly = p[keyyy].exterior.xy
    tmp = np.column_stack((x_poly, y_poly))
    points_converted = tform(tmp)
    poly_image_crs = shapely.geometry.LineString(points_converted)

    pol_conv[l] = poly_image_crs


erra = []
stda = []
for k in pol_conv_top.keys():
    if k == 'pol_full':
        continue
    err, std, p1, p2 = calc_err(pol_conv_top['pol_full'], pol_conv_top[k])
    print(k, err)
    erra.append(err)
    stda.append(std)

erras = []
stdas = []
for k in pol_conv.keys():
    if k == 'pol_full':
        continue
    err, std, p1, p2 = calc_err(pol_conv['pol_full'], pol_conv[k])
    print(k, err)
    erras.append(err)
    stdas.append(std)

# plt.errorbar(np.arange(0.1, 1, 0.1), erra, yerr=stda, fmt='o', capsize=5)

pol_full
pol_01
pol_02
pol_03
pol_04
pol_05
pol_06
pol_07
pol_08
pol_09
pol_half_top 0.7736234706559578
pol_half_bottom 1.1756789816687296
pol_half_left 3.368270462775243
pol_half_right 1.8690457312387114
pol_01 0.0
pol_02 0.0
pol_03 0.8753403681855582
pol_04 2.2009084439841766
pol_05 0.8376938851391224
pol_06 0.0
pol_08 0.8617195078034056


In [55]:
erra + erras

[0.7736234706559578,
 1.1756789816687296,
 3.368270462775243,
 1.8690457312387114,
 0.0,
 0.0,
 0.8753403681855582,
 2.2009084439841766,
 0.8376938851391224,
 0.0,
 0.8617195078034056]

In [259]:
for point in p1:
    plt.scatter(point.x, point.y, c='b')

plt.scatter(p1[0].x, p1[0].y, c='m')

for point in p2:
    plt.scatter(point.x, point.y, c='r')

plt.scatter(p2[0].x, p2[0].y, c='g')

In [149]:
fig, ax = plt.subplots()
ax.imshow(_image_stretch(rgb), extent=[georef[0], georef[0] + georef[1] * rgb.shape[1], georef[3] + georef[5] * rgb.shape[0], georef[3]])

# Make affine transformation matrix
aff_mat = np.array([[georef[1], georef[2], georef[0]],
                    [georef[4], georef[5], georef[3]],
                    [0, 0, 1]])

aff_mat_top = aff_mat.copy()
aff_mat_bottom = aff_mat.copy()

# convert georef to half top image
aff_mat_top[1, 2] = aff_mat[1, 2] + np.shape(rgb)[0]//2 * georef[5]

# convert georef to half bottom image
aff_mat_bottom[1, 2] = aff_mat[1, 2] + np.shape(rgb)[0]//2 * georef[5]

# Create affine transformation
tform = transform.AffineTransform(aff_mat)
tform_top = transform.AffineTransform(aff_mat_top)
tform_bottom = transform.AffineTransform(aff_mat_bottom)

# Transform polygon
x_poly, y_poly = pol_full['polygon_NIR_otsu'].exterior.xy
tmp = np.column_stack((x_poly, y_poly))
points_converted = tform(tmp)
poly_image_crs = shapely.geometry.Polygon(points_converted)

# Transform half top polygon
x_poly, y_poly = pol_half_top['polygon_NIR_otsu'].exterior.xy
tmp = np.column_stack((x_poly, y_poly))
points_converted = tform(tmp)
poly_image_crs_top = shapely.geometry.Polygon(points_converted)

# Transform half bottom polygon
x_poly, y_poly = pol_half_bottom['polygon_NIR_otsu'].exterior.xy
tmp = np.column_stack((x_poly[:-1], y_poly[:-1]))
points_converted = tform_bottom(tmp)
poly_image_crs_bottom = shapely.geometry.Polygon(points_converted)

gpd.GeoSeries(poly_image_crs.exterior).plot(ax=ax)
gpd.GeoSeries(poly_image_crs_top.exterior).plot(ax=ax, color='r')
gpd.GeoSeries(poly_image_crs_bottom.exterior).plot(ax=ax, color='g')

<Axes: >